# 13 Python建模库介绍

## 13.1 pandas与建模代码相结合

In [22]:
import pandas as pd
import numpy as np

data = pd.DataFrame({'x0': [1, 2, 3, 4, 5],
                     'x1': [0.01, -0.01, 0.25, -4.1, 0.],
                     'y': [-1.5, 0., 3.6, 1.3, -2.]})
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [23]:
data.columns

Index(['x0', 'x1', 'y'], dtype='object')

In [24]:
# 将DataFrame转化为Numpy，使用.values属性
data.values

array([[ 1.  ,  0.01, -1.5 ],
       [ 2.  , -0.01,  0.  ],
       [ 3.  ,  0.25,  3.6 ],
       [ 4.  , -4.1 ,  1.3 ],
       [ 5.  ,  0.  , -2.  ]])

In [25]:
df2 = pd.DataFrame(data.values, columns=['one', 'two', 'three'])
df2

,one,two,three
0,1.0,0.01,-1.5
1,2.0,-0.01,0.0
2,3.0,0.25,3.6
3,4.0,-4.10,1.3
4,5.0,0.00,-2.0


In [26]:
df3 = data.copy()
df3['strings'] = list('abcde')
df3.values

array([[1, 0.01, -1.5, 'a'],
       [2, -0.01, 0.0, 'b'],
       [3, 0.25, 3.6, 'c'],
       [4, -4.1, 1.3, 'd'],
       [5, 0.0, -2.0, 'e']], dtype=object)

In [27]:
# 只选择部分列
model_cols = ['x0', 'x1']
data.loc[:, model_cols].values

array([[ 1.  ,  0.01],
       [ 2.  , -0.01],
       [ 3.  ,  0.25],
       [ 4.  , -4.1 ],
       [ 5.  ,  0.  ]])

In [28]:
# 添加分类属性
data['category'] = pd.Categorical(list('abaab'), categories=['a', 'b'])

In [29]:
dummies = pd.get_dummies(data.category, prefix='category')
data_with_dummies = data.drop('category', axis=1).join(dummies)
data_with_dummies

,x0,x1,y,category_a,category_b
0,1,0.01,-1.5,1,0
1,2,-0.01,0.0,0,1
2,3,0.25,3.6,1,0
3,4,-4.10,1.3,1,0
4,5,0.00,-2.0,0,1


In [30]:
data

,x0,x1,y,category
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,a
3,4,-4.10,1.3,a
4,5,0.00,-2.0,b


In [21]:
data

,x0,x1,y,categories,category
0,1,0.01,-1.5,a,a
1,2,-0.01,0.0,b,b
2,3,0.25,3.6,a,a
3,4,-4.10,1.3,a,a
4,5,0.00,-2.0,b,b


## 13.2 使用Pasty创建模型描述

In [31]:
data = pd.DataFrame({'x0': [1, 2, 3, 4, 5],
                     'x1': [0.01, -0.01, 0.25, -4.1, 0, ],
                     'y': [1.5, 0., 3.6, 1.3, -2.1]})
data

,x0,x1,y
0,1,0.01,1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.1


In [32]:
import patsy

y, X = patsy.dmatrices('y~x0+x1', data)

In [33]:
y

DesignMatrix with shape (5, 1)
     y
   1.5
   0.0
   3.6
   1.3
  -2.1
  Terms:
    'y' (column 0)

In [34]:
X

DesignMatrix with shape (5, 3)
  Intercept  x0     x1
          1   1   0.01
          1   2  -0.01
          1   3   0.25
          1   4  -4.10
          1   5   0.00
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'x1' (column 2)

In [35]:
# 这些Patsy的DesignMatrix实例是含有附加元数据的Numpy ndarray
np.asarray(y)

array([[ 1.5],
       [ 0. ],
       [ 3.6],
       [ 1.3],
       [-2.1]])

In [36]:
np.asarray(X)

array([[ 1.  ,  1.  ,  0.01],
       [ 1.  ,  2.  , -0.01],
       [ 1.  ,  3.  ,  0.25],
       [ 1.  ,  4.  , -4.1 ],
       [ 1.  ,  5.  ,  0.  ]])

In [37]:
#Intercept(截距)名字的由来，这其实是线性模型，如最小二乘法回归中的惯例，可以通过给模型添加名词列+0来加入截距
patsy.dmatrices('y~x0 + x1 + 0', data)[1]

DesignMatrix with shape (5, 2)
  x0     x1
   1   0.01
   2  -0.01
   3   0.25
   4  -4.10
   5   0.00
  Terms:
    'x0' (column 0)
    'x1' (column 1)